In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np

df = pd.read_csv("CS105 W24 Survey (Responses) - Form Responses 1.csv")
#df
df2 = df.iloc[:, [0, 5, 7, 9]]
df2

,Timestamp,What gender do you identify as?,Who do you live with?,How many people live in your household?
0,2/9/2024 20:12:14,Male,Neither,6
1,2/9/2024 20:16:34,Female,Both,4
2,2/9/2024 20:18:55,Female,Friends,4
3,2/9/2024 20:24:00,Male,Neither,1
4,2/9/2024 20:26:16,Male,Neither,1
...,...,...,...,...
255,2/14/2024 19:46:28,Male,Friends,5
256,2/15/2024 0:28:38,Male,Family,North District 4 bed 2 bath
257,2/15/2024 8:33:45,Male,Family,9
258,2/15/2024 16:10:40,Female,Family,4
